### Constrative cross-encoder redaction test

In [1]:
import sys
sys.path.append('/home/jxm3/research/deidentification/unsupervised-deidentification')

In [2]:
from dataloader import WikipediaDataModule
import os

num_cpus = len(os.sched_getaffinity(0))

dm = WikipediaDataModule(
    document_model_name_or_path="roberta-base",
    profile_model_name_or_path="roberta-base",
    max_seq_length=128,
    dataset_name='wiki_bio',
    dataset_train_split='train[:256]', # not used in this notebook
    dataset_val_split='val[:20%]',
    dataset_version='1.2.0',
    word_dropout_ratio=0.0,
    word_dropout_perc=0.0,
    num_workers=1,
    num_nearest_neighbors=16,
    train_batch_size=64,
    eval_batch_size=64
)
dm.setup("fit")

Initializing WikipediaDataModule with num_workers = 1 and mask token `<mask>`
loading wiki_bio[1.2.0] split train[:256]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)


loading wiki_bio[1.2.0] split val[:20%]


Using custom data configuration default
Reusing dataset wiki_bio (/home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da)
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-wiki_bio_train__256__1.2.0__wiki.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-wiki_bio_val__20___1.2.0__wiki.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-wiki_bio_val__20___1.2.0__lexical_redacted.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-wiki_

Loading train nearest-neighbors from: /home/jxm3/research/deidentification/unsupervised-deidentification/nearest_neighbors/nn__train[:256]__256.p
Loading val nearest-neighbors from: /home/jxm3/research/deidentification/unsupervised-deidentification/nearest_neighbors/nn__val[:20%]__256.p


In [3]:
import glob
# glob.glob("../saves/cca__roberta__idf__n_8__dropout_-1.0_1.0_0.0/*/*_775/checkpoints/*.ckpt")
glob.glob("../saves/cca__roberta__idf__n_16__dropout_-1.0_1.0_0.0/*/*_777/checkpoints/*.ckpt")

['../saves/cca__roberta__idf__n_16__dropout_-1.0_1.0_0.0/deid-wikibio-4-cross-encoder_default/2jda7fn3_777/checkpoints/last.ckpt',
 '../saves/cca__roberta__idf__n_16__dropout_-1.0_1.0_0.0/deid-wikibio-4-cross-encoder_default/2jda7fn3_777/checkpoints/epoch=5-step=43703-idf_total.ckpt',
 '../saves/cca__roberta__idf__n_16__dropout_-1.0_1.0_0.0/deid-wikibio-4-cross-encoder_default/2jda7fn3_777/checkpoints/epoch=8-step=65555-idf_total.ckpt']

In [4]:
from model import ContrastiveCrossAttentionModel

import glob

# this model only trained with 1% of data -- here, after 19 epochs
# checkpoint_path =  glob.glob("../saves/cca__roberta__idf__n_8__dropout_-1.0_1.0_0.0/*/*_770/checkpoints/last.ckpt")[0]
# checkpoint_path = glob.glob("../saves/cca__roberta__idf__n_8__dropout_-1.0_1.0_0.0/*/*_775/checkpoints/last.ckpt")[0]
checkpoint_path = glob.glob("../saves/cca__roberta__idf__n_16__dropout_-1.0_1.0_0.0/*/*_777/checkpoints/last.ckpt")[0]

model = ContrastiveCrossAttentionModel.load_from_checkpoint(checkpoint_path)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initializing contrastive cross-encoder as document model (not creating profile model)
Initialized ContrastiveCrossAttention model with learning_rate = 2e-05 and patience 6


## 2. Define attack in TextAttack 

In [5]:
import textattack

### (a) Beam search + replace with `[MASK]`

In [6]:
class WordSwapSingleWord(textattack.transformations.word_swap.WordSwap):
    """Takes a sentence and transforms it by replacing with a single fixed word.
    """
    single_word: str
    def __init__(self, single_word: str = "?", **kwargs):
        super().__init__(**kwargs)
        self.single_word = single_word

    def _get_replacement_words(self, _word: str):
        return [self.single_word]

transformation = WordSwapSingleWord(single_word=dm.document_tokenizer.mask_token)
transformation(textattack.shared.AttackedText("Hello my name is Jack"))

[<AttackedText "<mask> my name is Jack">,
 <AttackedText "Hello <mask> name is Jack">,
 <AttackedText "Hello my <mask> is Jack">,
 <AttackedText "Hello my name <mask> Jack">,
 <AttackedText "Hello my name is <mask>">]

### (b) "Attack success" as fullfilment of the metric

In [7]:
from typing import List
import torch

class ChangeClassificationToBelowTopKClasses(textattack.goal_functions.ClassificationGoalFunction):
    k: int
    def __init__(self, *args, k: int = 1, **kwargs):
        self.k = k
        super().__init__(*args, **kwargs)

    def _is_goal_complete(self, model_output, _):
        original_class_score = model_output[self.ground_truth_output]
        num_better_classes = (model_output > original_class_score).sum()
        return num_better_classes >= self.k

    def _get_score(self, model_output, _):
        return 1 - model_output[self.ground_truth_output]
    
    
    """have to reimplement the following method to change the precision on the sum-to-one condition."""
    def _process_model_outputs(self, inputs, scores):
        """Processes and validates a list of model outputs.
        This is a task-dependent operation. For example, classification
        outputs need to have a softmax applied.
        """
        # Automatically cast a list or ndarray of predictions to a tensor.
        if isinstance(scores, list):
            scores = torch.tensor(scores)

        # Ensure the returned value is now a tensor.
        if not isinstance(scores, torch.Tensor):
            raise TypeError(
                "Must have list, np.ndarray, or torch.Tensor of "
                f"scores. Got type {type(scores)}"
            )

        # Validation check on model score dimensions
        if scores.ndim == 1:
            # Unsqueeze prediction, if it's been squeezed by the model.
            if len(inputs) == 1:
                scores = scores.unsqueeze(dim=0)
            else:
                raise ValueError(
                    f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
                )
        elif scores.ndim != 2:
            # If model somehow returns too may dimensions, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        elif scores.shape[0] != len(inputs):
            # If model returns an incorrect number of scores, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        elif not ((scores.sum(dim=1) - 1).abs() < 1e-4).all():
            # Values in each row should sum up to 1. The model should return a
            # set of numbers corresponding to probabilities, which should add
            # up to 1. Since they are `torch.float` values, allow a small
            # error in the summation.
            scores = torch.nn.functional.softmax(scores, dim=1)
            if not ((scores.sum(dim=1) - 1).abs() < 1e-4).all():
                raise ValueError("Model scores do not add up to 1.")
        return scores.cpu()


### (c) Wrapper for Wikipedia dataset

In [8]:
from typing import Optional, Tuple

from collections import OrderedDict

import datasets

class WikiDataset(textattack.datasets.Dataset):
    dataset: datasets.Dataset
    most_recent_idx: Optional[int]
    
    def __init__(self, dm: WikipediaDataModule):
        self.shuffled = True
        self.dataset = [ex for ex in dm.val_dataset]
        self.label_names = list(dm.val_dataset['name'])
        self.most_recent_idx = None
    
    def __len__(self) -> int:
        return len(self.dataset)
    
    def __getitem__(self, i: int) -> Tuple[OrderedDict, int]:
        input_dict = OrderedDict([
            ('document', self.dataset[i]['document'])
        ])
        self.most_recent_idx = i
        return input_dict, self.dataset[i]['text_key_id']
        

## (d) Model wrapper that computes similarities of input documents with validation profiles

In [9]:
from typing import Dict

import datasets
import numpy as np
import transformers

from model.model import Model

class MyModelWrapper(textattack.models.wrappers.ModelWrapper):
    model: Model
    tokenizer: transformers.AutoTokenizer
    max_seq_length: int
    # Have to store dataset and example_index here sadly.
    dataset: WikiDataset
    num_nearest_neighbors: int
    
    def __init__(self, model: Model, wiki_dataset: WikiDataset, tokenizer: transformers.AutoTokenizer, max_seq_length: int = 128, num_nearest_neighbors: int = 16):
        self.model = model
        self.model.eval()
        self.tokenizer = tokenizer
        self.wiki_dataset = wiki_dataset
        self.max_seq_length = max_seq_length
        self.num_nearest_neighbors = num_nearest_neighbors
                 
    def to(self, device):
        self.model.to(device)
        return self
    
    def _get_tokenized_profile(self, idx: int) -> Dict[str, torch.Tensor]:
        ex = self.wiki_dataset.dataset[idx]
        out_ex = {}
        for _k, _v in ex.items():
            if _k.startswith('profile__'):
                out_ex[_k.replace('profile__', '')] = torch.tensor([_v])
        return out_ex

    def _get_nearest_neighbors_tokenized(self) -> Tuple[List[int], Dict[str, torch.Tensor]]:
        """Gets the nearest-neighbors of an example. Used for contrastive learning."""
        ex = self.wiki_dataset.dataset[self.wiki_dataset.most_recent_idx]
        assert "nearest_neighbor_idxs" in ex
        out_ex = {}
        eligible_neighbor_idxs = [
            _i for _i in ex["nearest_neighbor_idxs"]
        ]
        assert len(eligible_neighbor_idxs) >= self.num_nearest_neighbors
        neighbor_idxs = eligible_neighbor_idxs[:self.num_nearest_neighbors]

        neighbors_tokenized = [
           self._get_tokenized_profile(idx=n_idx) for n_idx in neighbor_idxs if n_idx < len(self.wiki_dataset.dataset)
        ]
        keys = neighbors_tokenized[0].keys() # probably like {'input_ids', 'attention_mask'}
        for _k in keys:
            out_ex[_k] = torch.stack([
                n[_k][0] for n in neighbors_tokenized
            ])
        return neighbor_idxs, out_ex

    def __call__(self, text_input_list: List[str], batch_size=32):
        model_device = next(self.model.parameters()).device
        # TODO: Use bm25 on first input to get neighbor IDXs?
        
        # Tokenize document
        doc_inputs = self.tokenizer.batch_encode_plus(
            text_input_list,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        doc_inputs = {k: v for k,v in doc_inputs.items()}
        # print('doc_inputs.', doc_inputs['input_ids'].shape)
        
        # Get neighboring profiles
        neighbor_idxs, profile_inputs = self._get_nearest_neighbors_tokenized()
        num_neighbors = len(profile_inputs['input_ids'])
        
        # Do a loop. TODO: use repeats/tensors
        outputs = np.zeros((len(text_input_list), len(self.wiki_dataset.dataset)), dtype=np.float32)
        for idx in range(len(text_input_list)):
            doc_inputs_idx = { k: v[idx].repeat((num_neighbors, 1)) for k, v in doc_inputs.items() }
            # print('doc_inputs_idx', doc_inputs_idx['input_ids'].shape)
            # print('profile_inputs', profile_inputs['input_ids'].shape)
            all_inputs = {
                key: torch.cat((doc_inputs_idx[key], profile_inputs[key]), dim=1).to(model_device)
                for key in doc_inputs
            }
            with torch.no_grad():
                outputs_idx = self.model.forward_document_and_profile_inputs(inputs=all_inputs).squeeze()
            outputs[idx, neighbor_idxs] = outputs_idx.softmax(dim=0).cpu()
        return np.vstack(outputs)
            

In [10]:
wiki_dataset = WikiDataset(dm)
model_wrapper = MyModelWrapper(model=model, wiki_dataset=wiki_dataset, num_nearest_neighbors=16, tokenizer=dm.document_tokenizer, max_seq_length=dm.max_seq_length)
model_wrapper.to('cuda')

In [11]:
malcolm_mackey = dm.val_dataset[101]['document']
wiki_dataset[101] # access sets most_recent_idx
model_wrapper([malcolm_mackey]).argmax()

101

In [12]:
idx = 50

ex = dm.val_dataset[idx]
doc = ex['document']
wiki_dataset[idx] # access sets most_recent_idx
all_scores = model_wrapper([doc])
scores = all_scores[0, ex['nearest_neighbor_idxs']]

print('** printing nearest-neighbors **')
for idx, score in zip(model_wrapper.wiki_dataset.dataset[idx]['nearest_neighbor_idxs'][:16], scores.tolist()):
    print(f'\t\t{score*100:.2f}%', '\t',model_wrapper.wiki_dataset.dataset[idx]['name'])

** printing nearest-neighbors **
		100.00% 	 Anthony Acid
		0.00% 	 Dj Edwin
		0.00% 	 Dave Clarke -lrb- Techno Dj -rrb-
		0.00% 	 Dj Bobo
		0.00% 	 Dj Roxxi
		0.00% 	 Pmd
		0.00% 	 Anthony Johnson
		0.00% 	 Mclean
		0.00% 	 Anthony Caruso
		0.00% 	 3rd Bass
		0.00% 	 Anthony Stacchi
		0.00% 	 Drew Parks
		0.00% 	 Otto Knows
		0.00% 	 Lenny Fontana
		0.00% 	 Mike Dierickx
		0.00% 	 Amin Golestan Parast


## 3. Run attack once

In [13]:
class MaxNumWordsModified(textattack.constraints.PreTransformationConstraint):
    def __init__(self, max_num_words: int):
        self.max_num_words = max_num_words

    def _get_modifiable_indices(self, current_text):
        """Returns the word indices in current_text which are able to be
        modified."""

        if len(current_text.attack_attrs["modified_indices"]) >= self.max_num_words:
            return set()
        else:
            return set(range(len(current_text.words)))

    def extra_repr_keys(self):
        return ["max_num_words"]

In [14]:
from textattack.shared import utils


def get_modified_idxs_in_order(at: textattack.shared.AttackedText) -> List[int]:
    """Traverses linked-list of attacked texts from attack process
    and creates a list of the modified word indices.
    """
    modified_word_idxs = []
    while True:
        if 'newly_modified_indices' not in at.attack_attrs:
            break
        modified_word_idxs.extend(at.attack_attrs['newly_modified_indices'])
        at = at.attack_attrs['prev_attacked_text']
    modified_word_idxs = modified_word_idxs[::-1]
    return modified_word_idxs[::-1]


def diff_color_with_idxs(at: textattack.attack_results.AttackResult, color_method=None):
    """Highlights the difference between two texts using color.
    
    This version also adds idx numbers to show which words were masked in which order.

    Has to account for deletions and insertions from original text to
    perturbed. Relies on the index map stored in
    ``self.original_result.attacked_text.attack_attrs["original_index_map"]``.
    """
    t1 = at.original_result.attacked_text
    t2 = at.perturbed_result.attacked_text

    if color_method is None:
        return t1.printable_text(), t2.printable_text()

    color_1 = at.original_result.get_text_color_input()
    color_2 = at.perturbed_result.get_text_color_perturbed()

    # iterate through and count equal/unequal words
    words_1_idxs = []
    t2_equal_idxs = set()
    original_index_map = t2.attack_attrs["original_index_map"]
    for t1_idx, t2_idx in enumerate(original_index_map):
        if t2_idx == -1:
            # add words in t1 that are not in t2
            words_1_idxs.append(t1_idx)
        else:
            w1 = t1.words[t1_idx]
            w2 = t2.words[t2_idx]
            if w1 == w2:
                t2_equal_idxs.add(t2_idx)
            else:
                words_1_idxs.append(t1_idx)

    # words to color in t2 are all the words that didn't have an equal,
    # mapped word in t1
    words_2_idxs = list(sorted(set(range(t2.num_words)) - t2_equal_idxs))

    # make lists of colored words
    words_1 = [t1.words[i] for i in words_1_idxs]
    words_1 = [utils.color_text(w, color_1, color_method) for w in words_1]
    
    # First, replace words with `word_xx` where xx is the index
    # of the order that word was modified.
    word_modification_order = {word_idx: swap_idx+1 for swap_idx, word_idx in enumerate(get_modified_idxs_in_order(t2))}
    words_2 = [f'{t2.words[i]}__{word_modification_order[i]}' for i in words_2_idxs]
    words_2 = [utils.color_text(w, color_2, color_method) for w in words_2]

    t1 = at.original_result.attacked_text.replace_words_at_indices(
        words_1_idxs, words_1
    )
    t2 = at.perturbed_result.attacked_text.replace_words_at_indices(
        words_2_idxs, words_2
    )

    key_color = ("bold", "underline")
    return (
        t1.printable_text(key_color=key_color, key_color_method=color_method),
        t2.printable_text(key_color=key_color, key_color_method=color_method),
    )

In [15]:
from textattack.loggers import CSVLogger
from textattack.shared import AttackedText

import pandas as pd
class CustomCSVLogger(CSVLogger):
    """Logs attack results to a CSV."""

    def log_attack_result(self, result: textattack.goal_function_results.ClassificationGoalFunctionResult):
        # TODO print like 'mask1', 'mask2',
        original_text, perturbed_text = diff_color_with_idxs(result, color_method=self.color_method)
        original_text = original_text.replace("\n", AttackedText.SPLIT_TOKEN)
        perturbed_text = perturbed_text.replace("\n", AttackedText.SPLIT_TOKEN)
        result_type = result.__class__.__name__.replace("AttackResult", "")
        row = {
            "original_person": result.original_result._processed_output[0],
            "original_text": original_text,
            "original_text_id_bm25": bm25.get_scores(result.original_result.attacked_text.text.split()).argmax(),
            "perturbed_person": result.perturbed_result._processed_output[0],
            "perturbed_text": perturbed_text,
            "perturbed_text_id_bm25": bm25.get_scores(result.perturbed_result.attacked_text.text.split()).argmax(),
            "original_score": result.original_result.score,
            "perturbed_score": result.perturbed_result.score,
            "original_output": result.original_result.output,
            "perturbed_output": result.perturbed_result.output,
            "ground_truth_output": result.original_result.ground_truth_output,
            "num_queries": result.num_queries,
            "result_type": result_type,
        }
        self.df = pd.concat([self.df, pd.DataFrame([row])], ignore_index=True)
        self._flushed = False

In [16]:
from typing import List

from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi

eng_stopwords = stopwords.words('english')
from tqdm.auto import tqdm
tqdm.pandas()


def get_words_from_doc(s: List[str]) -> List[str]:
    words = s.split()
    return [w for w in words if not w in eng_stopwords]

def make_table_str(ex):
    ex['table_str'] = (
        ' '.join(ex['input_text']['table']['column_header'] + ex['input_text']['table']['content'])
    )
    return ex

prof_data = dm.val_dataset.map(make_table_str)
profile_corpus = prof_data['table_str']

tokenized_profile_corpus = [
    get_words_from_doc(prof) for prof in profile_corpus
]

bm25 = BM25Okapi(tokenized_profile_corpus)

Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/wiki_bio/default/1.2.0/c05ce066e9026831cd7535968a311fc80f074b58868cfdffccbc811dff2ab6da/cache-b236d6d839d77887.arrow


In [17]:
wiki_dataset = WikiDataset(dm)
model_wrapper = MyModelWrapper(model=model, wiki_dataset=wiki_dataset, num_nearest_neighbors=16, tokenizer=dm.document_tokenizer, max_seq_length=dm.max_seq_length)
model_wrapper.to('cuda')

In [18]:
# 
#  Initialize attack
# 

from textattack import Attack
from textattack.constraints.pre_transformation import MaxWordIndexModification, RepeatModification

goal_function = ChangeClassificationToBelowTopKClasses(model_wrapper, k=1)
constraints = [
    RepeatModification(),
    MaxWordIndexModification(max_length=dm.max_seq_length),
    MaxNumWordsModified(max_num_words=50)
]
transformation = WordSwapSingleWord(single_word=dm.document_tokenizer.mask_token)
search_method = textattack.search_methods.BeamSearch(beam_width=2)

attack = Attack(
    goal_function, constraints, transformation, search_method
)

from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs

attack_args = AttackArgs(num_examples=10, disable_stdout=True)

attacker = Attacker(attack, wiki_dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CustomCSVLogger(color_method='html')

# 
# Run attack
# 
from tqdm import tqdm

for result in results_iterable:
    tqdm._instances.clear() # Doesn't fix the progress bar :-(
    logger.log_attack_result(result)

from IPython.display import display, HTML

display(HTML(logger.df.to_html(escape=False)))

textattack: No entry found for goal function <class '__main__.ChangeClassificationToBelowTopKClasses'>.
textattack: Unknown if model of class <class 'model.contrastive_cross_attention.ContrastiveCrossAttentionModel'> compatible with goal function <class '__main__.ChangeClassificationToBelowTopKClasses'>.


Attack(
  (search_method): BeamSearch(
    (beam_width):  2
  )
  (goal_function):  ChangeClassificationToBelowTopKClasses
  (transformation):  WordSwapSingleWord
  (constraints): 
    (0): RepeatModification
    (1): MaxWordIndexModification(
        (max_length):  128
      )
    (2): MaxNumWordsModified(
        (max_num_words):  50
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [05:11<00:00, 31.18s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 40.79% |
| Average num. words per input: | 37.1   |
| Avg num queries:              | 746.1  |
+-------------------------------+--------+


textattack: Logging to CSV at path results.csv


,original_person,original_text,original_text_id_bm25,perturbed_person,perturbed_text,perturbed_text_id_bm25,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Michael iii of alexandria,"pope michael iii of alexandria ( also known as khail iii ) was the coptic pope of alexandria and patriarch of the see of st. mark ( 880 -- 907 ) .in 882 , the governor of egypt , ahmad ibn tulun , forced khail to pay heavy contributions , forcing him to sell a church and some attached properties to the local jewish community .this building was at one time believed to have later become the site of the cairo geniza .",0,Michael iv of alexandria,"pope <mask__1> <mask__4> of alexandria ( also known as khail iii ) was the coptic pope of alexandria and patriarch of the see of st. mark ( <mask__2> -- <mask__3> ) .in 882 , the governor of egypt , ahmad ibn tulun , forced khail to pay heavy contributions , forcing him to sell a church and some attached properties to the local jewish community .this building was at one time believed to have later become the site of the cairo geniza .",0,0.022335,0.825860,0,12300,0,500,Successful
1,Hui jun,hui jun is a male former table tennis player from china .,1,Jun marques davidson,<mask__11> <mask__10> <mask__6> <mask__5> <mask__2> <mask__9> <mask__3> <mask__4> <mask__7> <mask__1> <mask__8> .,2433,0.000008,0.763827,1,4393,1,122,Successful
2,Okan öztürk,okan Öztürk ( born 30 november 1977 ) is a turkish professional footballer .he currently plays as a striker for yeni malatyaspor .,2,Okan öztürk,<mask__19> <mask__20> ( <mask__3> <mask__2> <mask__1> <mask__18> ) <mask__6> <mask__7> <mask__14> <mask__4> <mask__13> .<mask__5> <mask__11> <mask__10> <mask__8> <mask__12> <mask__17> <mask__9> <mask__16> <mask__15> .,2433,0.000004,0.079084,2,2,2,401,Failed
3,Marie stephan,"marie stephan , ( born march 14 , 1996 ) is a professional squash player who represents france .she reached a career-high world ranking of world no. 101 in july 2015 .",3,Laura pomportes,"<mask__2> <mask__3> , ( born <mask__1> <mask__4> , <mask__5> ) is a professional squash player who represents france .<mask__8> reached a career-high world ranking of world no. <mask__7> in <mask__9> <mask__6> .",4624,0.004798,0.730643,3,4726,3,388,Successful
4,Leonard l. martino,leonard l. martino is a former democratic member of the pennsylvania house of representatives .he was born in butler to michael and angela pitullio martino .,4,Howard l. morgan,<mask__12> l. <mask__11> <mask__1> <mask__2> former <mask__5> <mask__9> <mask__4> the <mask__7> <mask__6> of <mask__3> .he was born in <mask__10> to michael and angela pitullio <mask__8> .,10833,0.000006,0.544473,4,6629,4,444,Successful
5,Salome jens,"salome jens ( born may 8 , 1935 ) is an american stage , film and television actress .she is perhaps best known for portraying the female changeling on '' '' .",5,Salome jens,"<mask__20> <mask__23> ( <mask__8> <mask__22> <mask__24> , <mask__21> ) <mask__18> <mask__19> <mask__5> <mask__11> , <mask__17> <mask__10> <mask__13> <mask__1> .<mask__15> <mask__14> <mask__7> <mask__6> <mask__2> <mask__4> <mask__16> <mask__12> <mask__3> <mask__25> <mask__9> '' '' .",11804,0.032871,0.236834,5,5,5,626,Failed
6,Carl crawford,"carl demonte crawford ( born august 5 , 1981 ) , nicknamed `` the perfect storm '' , is an american professional baseball left fielder with the los angeles dodgers of major league baseball ( mlb ) .he bats and throws left-handed .crawford was drafted by the tampa bay devil rays in the second round ( 52nd overall ) of the 1999 major league baseball draft .he made his major league debut in 2002 .crawford has more triples ( 121 ) than any other active baseball player .",6,Wade davis,"<mask__15> <mask__16> <mask__17> ( born august 5 , 1981 ) , nicknamed `` the perfect storm '' , is an american professional baseball <mask__7> <mask__14> <mask__4> the <mask__11> <mask__2> <mask__1> of major league baseball ( mlb ) .h